In [1]:
sys.executable

'/home/goonieb/anaconda2/bin/python'

In [15]:
import os
import json
import pymongo
import musicbrainzngs
import arrow
import re
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict, OrderedDict

from dateutil import parser
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [18]:
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
# setup mongo
client = MongoClient()
db = client["bandstour"]
venues =  db["selectedVenues"]
artists =  db["selectedArtists"]

def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.
    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).
    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
"(or 'y' or 'n').\n")

def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    s = re.sub('\s+', ' ', s)
    s = s.replace(' ', '-')

    s = re.sub('\W+', ' ', s)

    s = s.strip()

    s = s.lstrip()
    s = re.sub('\W+', '_', s)
    return s

#WE WANT TO LOOP OVER ARTISTS IN ORDER TO DEDUPLICATE THE RECORDS AND ASSIGN TOURS TO ONE ENTITY ONLY.
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose
for venueA in venues.find({},no_cursor_timeout=True) :
    
        
            #print "AAAA  =>", venueA["_id"]
            #print "AAAA slug  =>",slugify(venueA["_id"]).lower()
            for venueB in venues.find({},no_cursor_timeout=True) :
                if venueA["_id"]==venueB["_id"]:
                    pass
                elif (fuzz.ratio(\
                                        slugify(venueA["_id"]).lower()\
                                        , slugify(venueB["_id"]).lower()\
                                       ) > 95 and venueA["count"] >= venueB["count"]) :
                    print "AAAA  =>", venueA["_id"]
                    print "AAAA slug  =>",slugify(venueA["_id"]).lower()
                    print "AAAA gigs =>", venueA["count"]
                    
                    print "BBBB  =>",venueB["_id"]
                    print "BBBB slug  =>",slugify(venueB["_id"]).lower()
                    print "BBBB gigs =>", venueB["count"]
                    
                    print "_______________________________"
                    Answer_art= query_yes_no("MERGE "+venueB["_id"]+ " WITH "+ venueA["_id"]+" ?" , default="no")
                    print Answer_art
                    if Answer_art == True :
                        for artist in artists.find():
                            for gig   in artist["gigs"]:
                                if (gig["venue"]["name"]==venueB["_id"] and round(gig["venue"]["latitude"],3)== round(venueB["latitude"],3) and round(gig["venue"]["longitude"],3)== round(venueB["longitude"],3) ):
                                    print artist , gig["venue"]["name"]
                                    #artists.update({"_id" :artistA["_id"]}, artistA, upsert=True, multi=False)
                                    #artists.remove({"_id" :artistB["_id"]})
                            
                    

AAAA  => Seville Quarter-Duo
AAAA slug  => seville_quarter_duo
AAAA gigs => 2
BBBB  => Seville Quarter DUO
BBBB slug  => seville_quarter_duo
BBBB gigs => 2
_______________________________
MERGE Seville Quarter DUO WITH Seville Quarter-Duo ? [y/N] y
True
{u'count': 393, u'mbid': None, u'_id': u'Nick and the Ovorols', u'gigs': [{u'venue': {u'city': u'Chicago', u'name': u'Kingston Mines', u'country': u'United States', u'region': u'IL', u'longitude': -87.649221, u'place': u'Kingston Mines', u'latitude': 41.928694}, u'datetime': datetime.datetime(2012, 12, 5, 20, 0)}, {u'venue': {u'city': u'Chicago', u'name': u'Kingston Mines', u'country': u'United States', u'region': u'IL', u'longitude': -87.649221, u'place': u'Kingston Mines', u'latitude': 41.928694}, u'datetime': datetime.datetime(2012, 12, 12, 20, 0)}, {u'venue': {u'city': u'Chicago', u'name': u'Kingston Mines', u'country': u'United States', u'region': u'IL', u'longitude': -87.649221, u'place': u'Kingston Mines', u'latitude': 41.928694}

KeyboardInterrupt: 